# 🧪 PoC (Alumno) — AuraDB + OpenAI (NL→Cypher) con **mínimo preprocesamiento**
**Meta:** experimentar qué devuelve el modelo con un pipeline **muy básico** y reflexionar por qué hacen falta:
- **limpieza de fences** (```cypher …```),
- **bloqueo de escrituras** (CREATE/MERGE/SET/DELETE/LOAD CSV…),
- **validación** antes de ejecutar.

> Este cuaderno no incluye soluciones. Solo **pistas** y celdas con **TODO**.
**Última actualización:** 2025-09-24 22:04 UTC

## ⚠️ Aviso
- Usaremos `DRY_RUN = True` para **NO ejecutar** lo que devuelva el LLM.
- No cambies esto salvo que el profe lo indique explícitamente y el usuario sea **solo-lectura**.

## 1) Instalación rápida

In [ ]:
# TODO: instala dependencias (usa %pip)
# Pista: python-dotenv, neo4j, langchain, langchain-community, langchain-openai, tiktoken
# %pip install -q ...

## 2) Configuración (mínima)
Objetivo: cargar `.env`, crear **llm** (ChatOpenAI) y **graph** (Neo4jGraph).

**Pistas**
- `from dotenv import load_dotenv`; `load_dotenv()`
- Variables: `OPENAI_API_KEY`, `OPENAI_MODEL`, `NEO4J_URI`, `NEO4J_USERNAME`, `NEO4J_PASSWORD`
- `from langchain_openai import ChatOpenAI` + `os.environ["OPENAI_API_KEY"] = ...`
- `from langchain_community.graphs import Neo4jGraph`

In [ ]:
# TODO: completar configuración mínima
# import os
# from dotenv import load_dotenv
# load_dotenv()

# OPENAI_API_KEY = ...
# OPENAI_MODEL   = ...
# NEO4J_URI      = ...
# NEO4J_USERNAME = ...
# NEO4J_PASSWORD = ...

# from langchain_openai import ChatOpenAI
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
# llm = ChatOpenAI(model=OPENAI_MODEL, temperature=0)

# from langchain_community.graphs import Neo4jGraph
# graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)

# DRY_RUN = True
# print("DRY_RUN =", DRY_RUN)

## 3) Esquema del grafo (solo referencia)
**Pista:** `graph.refresh_schema()` y muestra un prefijo (`[:1200]`) de `graph.schema`.

In [ ]:
# TODO: refresca e imprime parte del esquema
# graph.refresh_schema()
# print(...)

## 4) Generación **naive** de Cypher
Propósito: **no** prohibir escrituras ni limpiar fences para ver qué pasa.

**Pistas**
- Crea un `PromptTemplate` con `input_variables=["schema","question"]`
- Template: "Genera una consulta Cypher ... Esquema: {schema} ... Pregunta: {question} ... Cypher:"
- Función `naive_generate_cypher(question)` que invoque el LLM con el prompt.

In [ ]:
# TODO: crear NAIVE_PROMPT y naive_generate_cypher
# from langchain.prompts import PromptTemplate
# NAIVE_PROMPT = PromptTemplate(...)
# def naive_generate_cypher(question: str) -> str:
#     ...

## 5) Ejecutor **naive** (imprime y, si se decide, ejecuta)
Por defecto, no ejecutaremos (DRY_RUN=True). Solo **imprime** el Cypher devuelto.

**Pistas**
- `naive_run(q)` debe:
  1) Obtener texto con `naive_generate_cypher(q)`
  2) Imprimirlo tal cual (posibles ``` o comandos peligrosos)
  3) Si `DRY_RUN` → devolver un dict con el texto y aviso
  4) Si no, ejecutar `graph.query(cy_text)` en `try/except`

In [ ]:
# TODO: implementar naive_run
# def naive_run(question: str):
#     ...

## 6) Demostración A — Caso **benigno** (lectura)
Pide algo sencillo, por ejemplo: *"Dame el top 5 de películas por rating."*

**Objetivo**
- Ver si el modelo devuelve una consulta limpia o con fences (```).
- Observar diferencias de estilo/estructura.

In [ ]:
# TODO: ejecuta naive_run con una pregunta de solo lectura
# out_a = naive_run("...")
# out_a

## 7) Demostración B — Caso **peligroso** (escritura)
Pide algo explícitamente destructivo, p. ej.: *"Crea una película de prueba..."*

**Objetivo**
- Comprobar si el LLM devuelve `CREATE/MERGE/SET/DELETE/...`.
- Entender por qué hay que **bloquear escrituras** antes de ejecutar.

## 8) Demostración C — Problema de fences (```cypher)
Muchos modelos envían la respuesta en un bloque con ``` que **rompe** la ejecución directa.

**Objetivo**
- Observar el problema y razonar cómo limpiarlo mínimamente.

In [ ]:
# TODO: ejecuta naive_run con otra pregunta de conteo (COUNT) y observa el formato
# out_c = naive_run("...")
# out_c

## 9) Alternativa **mínima** más segura (solo lectura + limpieza superficial)
**Sin dar la solución**, diseña dos funciones:
1) `strip_fences(text)` — Elimina ``` al inicio/fin y descarta la etiqueta inicial (p. ej., `cypher`).
   - Pista: `.strip()`, `.startswith("```")`, `.splitlines()`
2) `minimally_safe_generate(question)` — Usa tu naive, limpia con `strip_fences`, y **si** detectas palabras prohibidas (`CREATE|MERGE|SET|DELETE|REMOVE|DROP|LOAD CSV`), **lanza error** para proteger la BD.
3) `minimally_safe_run(question)` — Igual que `naive_run`, pero usando la versión “limpia y solo-lectura”.

In [ ]:
# TODO: implementar strip_fences, minimally_safe_generate y minimally_safe_run
# import re
# WRITE_RE = re.compile(..., re.IGNORECASE)
# def strip_fences(text: str) -> str:
#     ...
# def minimally_safe_generate(question: str) -> str:
#     ...
# def minimally_safe_run(question: str):
#     ...

## 10) Comparativa — naive vs. mínimo seguro
Usa **la misma pregunta** (p. ej., con keywords) y compara:
- ¿Hay diferencias en el Cypher final?
- ¿La versión segura bloquea algo? ¿Qué y por qué?

In [ ]:
# TODO: ejecuta naive_run y minimally_safe_run con la MISMA pregunta
# print("### NAIVE")
# naive = naive_run("...")
# print("\n### MIN SAFE")
# safe = minimally_safe_run("...")
# {"naive": naive, "safe": safe}

## 12) Conclusiones
- El LLM puede devolver **fences** y **escrituras** si no lo guiamos.
- Con dos pasos mínimos (quitar fences + bloquear escrituras) se evitan muchos problemas.
- En el cuaderno “definitivo” se añaden más protecciones (prompts estrictos, validadores, resumen controlado).

## 13) Documentación
- AuraDB — conectar apps: https://neo4j.com/docs/aura/connecting-applications/overview/
- Neo4j Python Driver: https://neo4j.com/docs/python-manual/current/
- LangChain (Neo4j): https://python.langchain.com/docs/integrations/graphs/neo4j
- Prompt Templates: https://python.langchain.com/docs/guides/prompt_templates/